In [1]:
import xarray
from dask.distributed import Client
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import xarray
import pandas as pd

from preprocess.sza import solarzenithangle


In [ ]:

client = Client(n_workers = 12)

In [ ]:
hres = xarray.open_zarr('/scratch/snx3000/kschuurm/DATA/HRSEVIRI.zarr')
hres = hres.sel(lat=slice(None, 62)).drop_vars(('lat_bnds', 'lon_bnds', 'record_status'))

In [ ]:
hres

In [ ]:
hres.to_zarr('HRSEVIRI.zarr', mode='w')

In [ ]:
sis = xarray.open_mfdataset('/scratch/snx3000/kschuurm/DATA/SARAH3/SIS_*.nc', 
                            concat_dim='time', 
                            combine='nested', 
                            data_vars='minimal', 
                            coords='minimal', 
                            compat='override', 
                            engine='h5netcdf')
sis = sis.drop_duplicates(dim='time').drop_vars(['lat_bnds', 'lon_bnds'])

intersec_tidx = sorted(
        list(
            set(hres.time.dt.round("30min").values).intersection(
                set(sis.time.dt.round("30min").values)
            )
        )
    )


hres_tidx_sort = hres.time.copy().sortby(hres.time)
hres_intersec_tidx = hres_tidx_sort.sel(time=intersec_tidx, method='nearest')

sis_reindex = sis.reindex(time=hres_intersec_tidx, lat=hres.lat, lon=hres.lon, method='nearest')
sis_reindex = sis_reindex.reindex(time=hres.time, fill_value=np.nan)
sis_reindex = sis_reindex.chunk({'time':60, 'lat':-1, 'lon':-1})
sis_reindex.to_zarr('HRSEVIRI.zarr', mode='a')


In [ ]:
hres= xarray.open_zarr('HRSEVIRI.zarr')
hres

In [ ]:
cal = xarray.open_mfdataset(['/scratch/snx3000/kschuurm/DATA/SARAH3/CAL_2014.nc','/scratch/snx3000/kschuurm/DATA/SARAH3/CAL_2015.nc'] , 
                            concat_dim='time', 
                            combine='nested', 
                            data_vars='minimal', 
                            coords='minimal', 
                            compat='override', 
                            engine='h5netcdf')

cal = cal.drop_duplicates(dim='time').drop_vars(['lat_bnds', 'lon_bnds','record_status'])

In [ ]:
intersec_tidx = sorted(
        list(
            set(hres.time.dt.round("30min").values).intersection(
                set(sis.time.dt.round("30min").values)
            )
        )
    )


hres_tidx_sort = hres.time.copy().sortby(hres.time)
hres_intersec_tidx = hres_tidx_sort.sel(time=intersec_tidx, method='nearest')

cal_reindex = cal.reindex(time=hres_intersec_tidx, lat=hres.lat, lon=hres.lon, method='nearest')
cal_reindex = cal_reindex.reindex(time=hres.time, fill_value=np.nan)
cal_reindex = cal_reindex.chunk({'time':60, 'lat':-1, 'lon':-1})
cal_reindex.to_zarr('HRSEVIRI.zarr', mode='a')


In [ ]:
DEM = xarray.open_dataset('/scratch/snx3000/kschuurm/DATA/SRTM/SRTM.nc')
roi= (62,35,-15,28)
SRTM = DEM.SRTM.isel(lat=(DEM.lat >= roi[1])&(DEM.lat<= roi[0]), lon=(DEM.lon >= roi[2])&(DEM.lon <= roi[3]))

weight_lat = int(np.ceil(len(SRTM.lat) / len(hres.lon)))
weight_lon = int(np.ceil(len(SRTM.lon)/ len(hres.lon)))
SRTM_coarse = SRTM.coarsen(lon=weight_lon, boundary='trim').mean().coarsen(lat=weight_lat, boundary='trim').mean()
SRTM_coarse = SRTM_coarse.reindex(lat=hres.lat, lon=hres.lon, method='nearest')

SRTM_coarse.attrs = {'long_name': 'Digital Elevation Map of the Shuttle Radar Topography Mission',
                     'standard_name': 'DEM_SRTM',
                     'units': 'm',
                    }

SRTM_coarse.to_zarr('HRSEVIRI.zarr',mode='a')

In [2]:
hres = xarray.open_zarr('/scratch/snx3000/kschuurm/DATA/HRSEVIRI.zarr')

In [3]:
hres

<xarray.Dataset>
Dimensions:        (time: 35966, lat: 503, lon: 801)
Coordinates:
  * lat            (lat) float64 35.03 35.08 35.13 35.19 ... 61.86 61.91 61.97
  * lon            (lon) float64 -14.97 -14.92 -14.87 ... 27.87 27.92 27.97
  * time           (time) datetime64[ns] 2014-01-01T07:27:00 ... 2015-12-31T1...
Data variables: (12/15)
    CAL            (time, lat, lon) float32 dask.array<chunksize=(60, 503, 801), meta=np.ndarray>
    SIS            (time, lat, lon) float32 dask.array<chunksize=(60, 503, 801), meta=np.ndarray>
    SRTM           (lat, lon) float64 dask.array<chunksize=(126, 401), meta=np.ndarray>
    channel_1      (time, lat, lon) float32 dask.array<chunksize=(60, 503, 801), meta=np.ndarray>
    channel_10     (time, lat, lon) float32 dask.array<chunksize=(60, 503, 801), meta=np.ndarray>
    channel_11     (time, lat, lon) float32 dask.array<chunksize=(60, 503, 801), meta=np.ndarray>
    ...             ...
    channel_5      (time, lat, lon) float32 dask.array<chunksize=(60, 503, 801), meta=np.ndarray>
    channel_6      (time, lat, lon) float32 dask.array<chunksize=(60, 503, 801), meta=np.ndarray>
    channel_7      (time, lat, lon) float32 dask.array<chunksize=(60, 503, 801), meta=np.ndarray>
    channel_8      (time, lat, lon) float32 dask.array<chunksize=(60, 503, 801), meta=np.ndarray>
    channel_9      (time, lat, lon) float32 dask.array<chunksize=(60, 503, 801), meta=np.ndarray>
    record_status  (time) int8 dask.array<chunksize=(60,), meta=np.ndarray>

In [6]:
datetimes = pd.to_datetime(hres.time)


def solarzenithangle_latlon(da_temp):
    subsample_int = 40
    da = xarray.zeros_like(da_temp[:, ::subsample_int, ::subsample_int]) 
    
    da.attrs.update({'long_name': 'Solar Zenith Angle at sea level',
                      'standard_name': 'solar_zenith_angle',
                      'units':'rad'})
    datetimes = pd.to_datetime(da.time)
    for i, lat in enumerate(da.lat[::subsample_int]):
        for j, lon in enumerate(da.lon[::subsample_int]):
            da[:,i, j] = solarzenithangle(datetimes, lat.item(), lon.item(), 0)
    
    da = da.interp(coords=da_temp, method='cubic')
    return da


        

In [7]:
SZA = solarzenithangle_latlon(hres.SIS)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
SZA

In [ ]:
SZA.to_zarr('/scratch/snx3000/kschuurm/DATA/HRSEVIRI.zarr',mode='a')

2023-12-16 15:30:25,007 - distributed.nanny - WARNING - Restarting worker
/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 13.69 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2023-12-16 15:32:54,485 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:41941
Traceback (most recent call last):
  File "/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/distributed/comm/tcp.py", line 547, in connect
    stream = await self.client.connect(
  File "/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/tornado/tcpclient.py", line 279, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/python/3.9.4.1/lib/python3.9/asyn